# Chatbot with RAG

## 1. env setting

### 1.1 pip install

In [2]:
# requirements.txt에 따른 dep 설치

%pip install -r requirements.txt

Processing c:\home\conda\feedstock_root\build_artifacts\asttokens_1698341106958\work (from -r requirements.txt (line 15))
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\home\\conda\\feedstock_root\\build_artifacts\\asttokens_1698341106958\\work'



In [1]:
%pip install streamlit


  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.1-py3-none-any.whl.metadata (4.3 kB)
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   -------------- ------------------------- 3.1/8.7 MB 15.4 MB/s eta 0:00:01
   ----------------------------- ---------- 6.6/8.7 MB 16.1 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 16.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/658.1 kB ? eta -:--:--
   ---------------------------------------- 658.1/658.1 kB 8.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 2.6/2.6 MB 18.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ---------------- ----------------------- 2.9/6.9 MB 13.9 MB/s eta 0:00:01
   ------------------------------------- -- 6.6/6.9 MB 15.5 MB/s eta 0:00:01
   ---------------------------------------- 6.9/6.9 MB

### 1.2 import

In [11]:
import os, getpass, pprint

from langchain_chroma import Chroma
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain.docstore.document import Document
from langchain_text_splitters import Language, RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import streamlit as st


In [10]:
if "UPSTAGE_API_KEY" not in os.environ:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")

solar_api_key=os.environ["UPSTAGE_API_KEY"]

solar_api_key[:3]

'up_'

## 2. Embedding을 통한 DB 제작

In [7]:
embeddings = UpstageEmbeddings(
  api_key=solar_api_key,
  model="solar-embedding-1-large"
)

In [8]:
# test

doc_result = embeddings.embed_documents(
    ["Sam is a teacher.", "This is another document"]
)
print(doc_result)
 
query_result = embeddings.embed_query("What does Sam do?")
print(query_result)

[[0.016357421875, 0.0171356201171875, -0.0077972412109375, 0.024078369140625, 0.00342559814453125, -0.00682830810546875, -0.01465606689453125, -0.01067352294921875, -0.015625, 0.00673675537109375, 0.0183258056640625, 0.00507354736328125, 0.007244110107421875, 0.01177978515625, 0.02777099609375, 0.02191162109375, -0.0213623046875, -0.0012531280517578125, -0.0030117034912109375, -0.0167388916015625, -0.02410888671875, -0.0090789794921875, -0.01122283935546875, -0.005924224853515625, -0.01036834716796875, 0.0162811279296875, 0.002777099609375, -0.01197052001953125, 0.00518798828125, 0.02154541015625, 0.0056610107421875, 0.01412200927734375, 0.0020542144775390625, -0.0094146728515625, 0.01070404052734375, -0.01226806640625, -0.003635406494140625, 0.02374267578125, -0.012420654296875, 0.01287078857421875, -0.0112152099609375, -0.02001953125, -0.01410675048828125, 8.320808410644531e-05, -0.016510009765625, 0.012786865234375, 0.00691986083984375, -0.0008373260498046875, 0.004840850830078125, 

In [8]:
sample_text="""
ECO2001(구 경제10) 경제학원론 Ⅰ(강의 3시간) 3학점 본 강좌는 소비자 이론과 생산자 이론을 중심으로 게임이론, 산업조직, 노동경제학의 주요 문제들을 접목하여 미시 경제학에서 기초가 되는 이론들을 공부한다. ECO2002(구 경제11) 경제학원론 Ⅱ(강의 3시간) 3학점 경제 분석의 기초적인 방법 중에서 거시경제학에 해당하는 것을 공부한다. 국민소득 계정 및 개념, 균형국민소득의 결정, 화폐와 은행제도, 인플레이션과 실업, 성장과 발전, 국제경제학의 기초이론을 다룬다. ECO2003 경제수리기초(강의 3시간) 3학점 경제이론과 통계학을 이해하기 위해 필요한 행렬과 벡터, 미분과 적분의 개념을 소개한다. 또한 경제 모형의 분석에 필요한 편미분과 최적화 문제의 해법을 다룬다. 엄격한 증명보다는 경제학 문제들을 통하여 수학의 실제적인 적용 방법을 학습하는 것을 목적으로 한다. ECO2004(구 경제40) 경제통계학(강의 3시간) 3학점 도수분포, 확률분포, 추정과 검정 등 기술통계학 및 추리통계학의 기초이론을 다룬다. 또한 경제모형의 실증분석에 사용되는 상관관계와 회귀분석의 개념 및 응용에 대해 공부한다.(경제학을 제2, 3전공으로 이수하는 경우 경영통계학, 통계학으로 대체할 수 있으며 이 중에서 한 과목만을 학점이수로 인정) ECO2005(구 경제50) 경제정보분석(강의 및 실습 3시간) 3학점 선수과목：ECO2004(혹은 MGT2002나 MAT3020) 경제통계학에서 배운 통계·계량 분석기법 및 이론을 현실 경제 통계자료에 접목시키는 방법을 배운다. PC 입문을 위한 기본적인 컴퓨터 실습 이외에 여러 가지 통계 기법을 자료분석에 직접 적용시키는 경제 통계학 및 계량경제학의 Package, Program 사용법을 다룬다. ECO2006(구 경제70) 미시경제학 Ⅰ(강의 3시간) 3학점 선수과목：ECO2001, ECO2003(혹은 STS2005나 STS2006) 시장경제 하에서 자원배분, 소득분배가 결정되는 원리를 배운다. 수요와 공급의 개념, 소비자균형과 시장수요, 생산과 생산비의 이론, 생산물시장과 생산요소시장의 균형, 그리고 일반균형·후생경제학의 기초 이론을 다룬다. ECO2007(구 경제80) 거시경제학 Ⅰ(강의 3시간) 3학점 선수과목：ECO2001, ECO2002, ECO2003(혹은 STS2005나 STS2006) 국가 경제의 여러 현상, 특히 국민소득의 변동, 경제성장, 인플레이션, 실업률 등을 이해하기 위해 집계변수들의 동향을 파악하고 이에 대한 분석의 틀을 제공하는 한편, 경기변동을 완화하고 지속적인 경제성장을 달성하기 위한 정부의 정책 수단과 방법, 효과, 한계 등을 논의한다."""

In [9]:
sample_list = sample_text.split(".")[:-1]
sample_list

['\nECO2001(구 경제10) 경제학원론 Ⅰ(강의 3시간) 3학점 본 강좌는 소비자 이론과 생산자 이론을 중심으로 게임이론, 산업조직, 노동경제학의 주요 문제들을 접목하여 미시 경제학에서 기초가 되는 이론들을 공부한다',
 ' ECO2002(구 경제11) 경제학원론 Ⅱ(강의 3시간) 3학점 경제 분석의 기초적인 방법 중에서 거시경제학에 해당하는 것을 공부한다',
 ' 국민소득 계정 및 개념, 균형국민소득의 결정, 화폐와 은행제도, 인플레이션과 실업, 성장과 발전, 국제경제학의 기초이론을 다룬다',
 ' ECO2003 경제수리기초(강의 3시간) 3학점 경제이론과 통계학을 이해하기 위해 필요한 행렬과 벡터, 미분과 적분의 개념을 소개한다',
 ' 또한 경제 모형의 분석에 필요한 편미분과 최적화 문제의 해법을 다룬다',
 ' 엄격한 증명보다는 경제학 문제들을 통하여 수학의 실제적인 적용 방법을 학습하는 것을 목적으로 한다',
 ' ECO2004(구 경제40) 경제통계학(강의 3시간) 3학점 도수분포, 확률분포, 추정과 검정 등 기술통계학 및 추리통계학의 기초이론을 다룬다',
 ' 또한 경제모형의 실증분석에 사용되는 상관관계와 회귀분석의 개념 및 응용에 대해 공부한다',
 '(경제학을 제2, 3전공으로 이수하는 경우 경영통계학, 통계학으로 대체할 수 있으며 이 중에서 한 과목만을 학점이수로 인정) ECO2005(구 경제50) 경제정보분석(강의 및 실습 3시간) 3학점 선수과목：ECO2004(혹은 MGT2002나 MAT3020) 경제통계학에서 배운 통계·계량 분석기법 및 이론을 현실 경제 통계자료에 접목시키는 방법을 배운다',
 ' PC 입문을 위한 기본적인 컴퓨터 실습 이외에 여러 가지 통계 기법을 자료분석에 직접 적용시키는 경제 통계학 및 계량경제학의 Package, Program 사용법을 다룬다',
 ' ECO2006(구 경제70) 미시경제학 Ⅰ(강의 3시간) 3학점 선수과목：ECO2001, ECO2003(혹은 STS2005나 STS2006) 시장경제 하에서

In [16]:
#create document
splits = RecursiveCharacterTextSplitter().create_documents(sample_list)

pprint.pprint(splits)

[Document(metadata={}, page_content='ECO2001(구 경제10) 경제학원론 Ⅰ(강의 3시간) 3학점 본 강좌는 소비자 이론과 생산자 이론을 중심으로 게임이론, 산업조직, 노동경제학의 주요 문제들을 접목하여 미시 경제학에서 기초가 되는 이론들을 공부한다'),
 Document(metadata={}, page_content='ECO2002(구 경제11) 경제학원론 Ⅱ(강의 3시간) 3학점 경제 분석의 기초적인 방법 중에서 거시경제학에 해당하는 것을 공부한다'),
 Document(metadata={}, page_content='국민소득 계정 및 개념, 균형국민소득의 결정, 화폐와 은행제도, 인플레이션과 실업, 성장과 발전, 국제경제학의 기초이론을 다룬다'),
 Document(metadata={}, page_content='ECO2003 경제수리기초(강의 3시간) 3학점 경제이론과 통계학을 이해하기 위해 필요한 행렬과 벡터, 미분과 적분의 개념을 소개한다'),
 Document(metadata={}, page_content='또한 경제 모형의 분석에 필요한 편미분과 최적화 문제의 해법을 다룬다'),
 Document(metadata={}, page_content='엄격한 증명보다는 경제학 문제들을 통하여 수학의 실제적인 적용 방법을 학습하는 것을 목적으로 한다'),
 Document(metadata={}, page_content='ECO2004(구 경제40) 경제통계학(강의 3시간) 3학점 도수분포, 확률분포, 추정과 검정 등 기술통계학 및 추리통계학의 기초이론을 다룬다'),
 Document(metadata={}, page_content='또한 경제모형의 실증분석에 사용되는 상관관계와 회귀분석의 개념 및 응용에 대해 공부한다'),
 Document(metadata={}, page_content='(경제학을 제2, 3전공으로 이수하는 경우 경영통계학, 통계학으로 대체할 수 있으며 이 중에서 한 과목만을 학점이수로 인정) ECO2005

In [17]:
# create vectorstore

vectorstore = Chroma.from_documents(
    collection_name="sogang_vectorDB",
    documents=splits,
    ids=[doc.page_content for doc in splits],
    embedding=UpstageEmbeddings(model="solar-embedding-1-large"),
    persist_directory='./DB_sameple'
)

In [26]:
# Load chromaDB 
vectorstore = Chroma(
    collection_name="sogang_vectorDB",
    embedding_function=UpstageEmbeddings(model="solar-embedding-1-large"),
    persist_directory="./DB"
    )
vectorstore

In [31]:
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={'k':2})

In [32]:
answer=retriever.invoke("서강대학교 위치")

Number of requested results 20 is greater than number of elements in index 4, updating n_results = 4


In [33]:
answer

[Document(metadata={}, page_content='서강대학교는 서울 마포구에 위치한 대한민국의 명문 사립대학으로, 1960년에 설립되었습니다'),
 Document(metadata={}, page_content='우수한 교수진과 연구 환경을 갖추고 있으며, 산학협력을 통해 다양한 산업과 연계한 교육 프로그램을 제공합니다')]

## 3. prompt를 통한 텍스트 생성

In [ ]:
def chat_with_prompt():

# langchain, 1. llm define, 2. prompt define, 3. chain, 4. chain.invoke

    # 1. define your favorate llm, solar
    llm = ChatUpstage()

    # 2. define chat prompt
    rag_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "What is the capital of France?"),
        ("ai", "I know of it. It's Paris!!"),
        ("human", "What about Korea?"),
    ]
)

    # 3. define chain
    chain = rag_with_history_prompt | llm | StrOutputParser()

# 4. invoke the chain
c_result = chain.invoke({})
print(c_result)

## 4. streamlit으로 출력

In [4]:
with st.sidebar:
    openai_api_key = st.text_input("OpenAI API Key", key="chatbot_api_key", type="password")
    "[Get an OpenAI API key](https://platform.openai.com/account/api-keys)"
    "[View the source code](https://github.com/streamlit/llm-examples/blob/main/Chatbot.py)"
    "[![Open in GitHub Codespaces](https://github.com/codespaces/badge.svg)](https://codespaces.new/streamlit/llm-examples?quickstart=1)"

st.title("Sogang-i(서강이)")
st.caption("🚀 A Streamlit chatbot powered by SOLAR-mini(Upstage)")
if "messages" not in st.session_state:
    st.session_state["messages"] = [{"role": "assistant", "content": "How can I help you?"}]

for msg in st.session_state.messages:
    st.chat_message(msg["role"]).write(msg["content"])

if prompt := st.chat_input():
    if not solar_api_key:
        st.info("Please add your SOLAR API key to continue.")
        st.stop()

    client = 
    st.session_state.messages.append({"role": "user", "content": prompt})
    st.chat_message("user").write(prompt)
    response = client.chat.completions.create(model="gpt-3.5-turbo", messages=st.session_state.messages)
    msg = response.choices[0].message.content
    st.session_state.messages.append({"role": "assistant", "content": msg})
    st.chat_message("assistant").write(msg)

SyntaxError: invalid syntax (3570495215.py, line 20)